In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
df = pd.DataFrame.from_records(bb.get_drink_items()[0], columns = ['drink', 'count'])
df['business_id'] = 'vCEc52697fPzYGh_bzS29A'
df = bb.df_business[['business_id', 'name', 'address', 'city', 'state', 'overall_star', 'review_count']].merge(df)
df = df.rename(columns = {'drink': 'drink_name', 'count': 'drink_count'})

from sqlalchemy import create_engine
engine = create_engine("sqlite:///yelp.db", echo=True)
df.to_sql("top_drinks", con=engine, if_exists="replace")

2021-12-07 23:15:07,320 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("top_drinks")
2021-12-07 23:15:07,321 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-12-07 23:15:07,322 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("top_drinks")
2021-12-07 23:15:07,322 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-12-07 23:15:07,323 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-12-07 23:15:07,323 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-12-07 23:15:07,324 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("top_drinks")
2021-12-07 23:15:07,324 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-12-07 23:15:07,325 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-12-07 23:15:07,325 INFO sqlalchemy.engine.Engine [raw sql] ('top_drinks',)
2021-12-07 23:15:07,326 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_li

In [99]:
table_name = 'top_drinks'
index_column = 'business_id'

import sqlite3
db = sqlite3.connect('yelp.db')
cursor = db.cursor()
sql = f"CREATE INDEX index_{table_name} ON {table_name} ({index_column});"
cursor.execute(sql)

In [100]:
%%time
from sqlalchemy import create_engine
engine = create_engine("sqlite:///yelp.db", echo=False)
query = f"""SELECT * from top_drinks
            """
pd.read_sql(
    query,
    con=engine,
)

CPU times: user 3.91 ms, sys: 938 µs, total: 4.85 ms
Wall time: 4.24 ms


,index,business_id,name,address,city,state,overall_star,review_count,drink_name,drink_count
0,0,vCEc52697fPzYGh_bzS29A,Gong Cha,"75 Middlesex Turnpike, Ste 2115",Burlington,MA,3.5,36,oolong milk tea,2
1,1,vCEc52697fPzYGh_bzS29A,Gong Cha,"75 Middlesex Turnpike, Ste 2115",Burlington,MA,3.5,36,caramel milk tea,2
2,2,vCEc52697fPzYGh_bzS29A,Gong Cha,"75 Middlesex Turnpike, Ste 2115",Burlington,MA,3.5,36,brown sugar milk tea,2
3,3,vCEc52697fPzYGh_bzS29A,Gong Cha,"75 Middlesex Turnpike, Ste 2115",Burlington,MA,3.5,36,pearl milk tea,2
4,4,vCEc52697fPzYGh_bzS29A,Gong Cha,"75 Middlesex Turnpike, Ste 2115",Burlington,MA,3.5,36,mango slush,2
...,...,...,...,...,...,...,...,...,...,...
1179,1179,vCEc52697fPzYGh_bzS29A,Gong Cha,"75 Middlesex Turnpike, Ste 2115",Burlington,MA,3.5,36,large milk tea,1
1180,1180,vCEc52697fPzYGh_bzS29A,Gong Cha,"75 Middlesex Turnpike, Ste 2115",Burlington,MA,3.5,36,earl grey milk tea,1
1181,1181,vCEc52697fPzYGh_bzS29A,Gong Cha,"75 Middlesex Turnpike, Ste 2115",Burlington,MA,3.5,36,traditional bubble tea,1
1182,1182,vCEc52697fPzYGh_bzS29A,Gong Cha,"75 Middlesex Turnpike, Ste 2115",Burlington,MA,3.5,36,strawberry green tea,1


In [15]:
import pandas as pd
import numpy as np
import ast
from sqlalchemy import create_engine

In [16]:
boston_boba = pd.read_csv('data/boston_boba_reviews.csv')

In [6]:
boston_boba.name.unique()

array(['Dumpling Daughter', 'Kung Fu Tea', 'Hearty Teriyaki Newton',
       'Boba Me', "Patty Chen's Dumpling Room", 'Boston Tea Stop',
       'Tsaocaa', 'Chatime', 'Mala Kungfu', "Beard Papa's",
       'M&J Teriyaki', 'Tea Bar by Evy', 'Happy Cafe',
       'My Thai Vegan Cafe', 'Yotopia', 'Sweet Heart', 'Pholicious',
       'Kirin Teppanyaki & Wok', 'Crepe & Ice Cream 89C', 'Bubble Nation',
       'Golden Sea Bubble Tea & Ice Cream', 'Tbaar', 'Sweet Kingdom',
       'One Zo', 'Limoo Tea Bar', 'Jaho Coffee & Tea',
       '4 Seasons Boba Tea', 'Fruity', 'Tasty Pot', 'Lollicup',
       'Love Art Sushi', 'Sweet Boba', 'Sweet Waffles + Boba',
       'Thana Thai Kitchen', 'Saigon Fusion', 'Shinmio Tea', 'Gong Cha',
       'Dizin FruttiBerri', 'YoCha', 'Vivi Bubble Tea Cafe',
       'Reign Drink Lab', 'Royaltea - Brookline', 'Real Lucky',
       'Vivi Bubble Tea Cafe Quincy', 'Pin Ming', 'The Juice Bar',
       'Dolphin Bay', 'Vivi Bubble Tea Brighton', 'OneZo',
       'Happy Lemon Boston', 

In [7]:
%%time
from collections import defaultdict
import spacy
from collections import Counter
nlp = spacy.load('en_core_web_sm')

CPU times: user 495 ms, sys: 44.9 ms, total: 540 ms
Wall time: 581 ms


In [97]:
from boba_business import BobaBusiness
bb = BobaBusiness('Gong Cha')
bb.get_drink_items()[0]

[('oolong milk tea', 2),
 ('caramel milk tea', 2),
 ('brown sugar milk tea', 2),
 ('pearl milk tea', 2),
 ('mango slush', 2),
 ('quality bubble tea', 1),
 ('green tea', 1),
 ('really really good bubble tea', 1),
 ('great milk foam tea', 1),
 ('original milk tea', 1),
 ('really good bubble tea', 1),
 ('an earl grey milk tea', 1),
 ('a large passion fruit green tea', 1),
 ('an awesome bubble tea', 1),
 ('taro milk tea', 1),
 ('carmel milk tea', 1),
 ('classic- pearl milk tea', 1),
 ('red plum tea', 1),
 ('two taro milk tea', 1),
 ('good bubble tea', 1),
 ('oolong tea', 1),
 ('green milk tea', 1),
 ('a bit overpriced bubble tea', 1),
 ('early gray milk tea', 1),
 ('2nd caramel milk tea', 1),
 ('some high quality bubble tea', 1),
 ('some authentic bubble tea', 1),
 ('large milk tea', 1),
 ('earl grey milk tea', 1),
 ('traditional bubble tea', 1),
 ('strawberry green tea', 1),
 ('mango soy milk slush', 1)]

In [24]:
bb.get_keyword_items(['boba'])[0]

[('boba', 22),
 ('the boba', 12),
 ('good boba', 2),
 ('my boba', 2),
 ('perfectly chewy boba', 1),
 ('your boba', 1),
 ('the black sugar boba', 1),
 ("me's boba", 1),
 ('the best boba', 1),
 ('what good boba', 1),
 ('such good boba', 1),
 ('really good boba', 1),
 ('yummy boba', 1),
 ('best boba', 1),
 ('those boba', 1),
 ('their boba', 1),
 ('"peace, love, boba', 1),
 ('a huge boba', 1),
 ('the taro boba', 1)]

In [20]:
bb.noun_counts.most_common()

[('i', 118),
 ('it', 67),
 ('boba', 53),
 ('tea', 39),
 ('you', 28),
 ('they', 27),
 ('me', 25),
 ('place', 24),
 ('drink', 17),
 ('area', 17),
 ('drinks', 17),
 ('which', 15),
 ('we', 11),
 ('that', 10),
 ('hill', 10),
 ('sugar', 8),
 ('spot', 8),
 ('flavors', 8),
 ('boston', 8),
 ('games', 7),
 ('texture', 7),
 ('shop', 7),
 ('staff', 7),
 ('what', 6),
 ('people', 6),
 ('all', 6),
 ('cups', 6),
 ('cup', 6),
 ('places', 6),
 ('employees', 6),
 ('this', 6),
 ('day', 5),
 ('who', 5),
 ('matcha', 5),
 ('time', 5),
 ('ice', 5),
 ('decor', 5),
 ('milk', 5),
 ('mi', 5),
 ('sweetness', 4),
 ('space', 4),
 ('jenga', 4),
 ('bit', 4),
 ('chewy', 4),
 ('fries', 4),
 ('some', 4),
 ('things', 4),
 ('menu', 4),
 ('interior', 4),
 ('friends', 4),
 ('them', 4),
 ('she', 4),
 ('neighborhood', 4),
 ('side', 3),
 ('game', 3),
 ('price', 3),
 ('taro', 3),
 ('those', 3),
 ('amount', 3),
 ('prices', 3),
 ('jelly', 3),
 ('store', 3),
 ('restaurant', 3),
 ('pictures', 3),
 ('students', 3),
 ('california', 3)

In [266]:
bb.weighted_noun_scores.most_common(20)

[('sandwich', 212314225.05307856),
 ('boba', 69828722.00263505),
 ('chewiness', 49019607.84313726),
 ('mi', 39062500.0),
 ('chewy', 16393442.622950818),
 ('jenga', 15209125.47528517),
 ('ubereats', 13175230.566534914),
 ('fries', 13029315.960912053),
 ('lychee', 11235955.056179775),
 ('matcha', 10683760.683760684),
 ('shop', 9320905.459387483),
 ('carrot', 9174311.926605504),
 ('syrupy', 8547008.547008546),
 ('meals', 7874015.748031495),
 ('serve', 7812499.999999999),
 ('teas', 6293706.293706293),
 ('allston', 5128205.128205128),
 ('slush', 4338394.793926247),
 ('oolong', 4166666.666666667),
 ('options', 4032258.064516129)]